In [7]:
#设置这个选项打印所有的值,如果不使用Jupytor Notebook，请注释下面2行
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
 
from pyspark.ml.recommendation import ALS


df = spark.createDataFrame(
    [(0, 0, 4.0), (0, 1, 2.0), (1, 1, 3.0), (1, 2, 4.0), 
     (2, 1, 1.0), (2, 2, 5.0), (0, 3, 3.0), (1, 3, 4.0)],
    ["user", "item", "rating"])
# rank 特征数量k=5
als = ALS(rank=5, maxIter=5, seed=0)
model = als.fit(df)
 
print('# 用户特征向量U')
model.userFactors.orderBy("id").collect()
 
print('# 物品特征向量V')
model.itemFactors.orderBy("id").collect()
print("\n") 
 
test = spark.createDataFrame([(0, 2), (1, 0), (2, 0),(2, 3)], ["user", "item"])


predictions = sorted(model.transform(test).collect(), key=lambda r: r[0])
print('# 对于未知的用户物品推荐度预测结果')
for i in predictions:
    print(i)

print("\nU0用户对所有物品的评分")
user_recs = model.recommendForAllUsers(4) 
print(user_recs.where(user_recs.user == 0).select("recommendations.item", "recommendations.rating").collect())
 
print("\n所有用户对物品V2的评分" ) 
item_recs = model.recommendForAllItems(3)
print(item_recs.where(item_recs.item == 2).select("recommendations.user", "recommendations.rating").collect())
 
print("\n根据mode1分解的两个矩阵，对用户U2子集中3个物品，进行评分预测，然后与源数据对比")  
user_subset = df.where(df.user == 2)
user_subset_recs = model.recommendForUserSubset(user_subset, 3)
print(user_subset_recs.select("recommendations.item", "recommendations.rating").first())

print("\n根据mode1分解的两个矩阵，对物品V0子集中3个用户，进行评分预测，然后与源数据对比") 
item_subset = df.where(df.item == 0)
item_subset_recs = model.recommendForItemSubset(item_subset, 3)
print(item_subset_recs.select("recommendations.user", "recommendations.rating").first())
 


# 用户特征向量U


[Row(id=0, features=[0.5617156624794006, -0.44229641556739807, 0.34542036056518555, 0.905699610710144, 0.4823412597179413]),
 Row(id=1, features=[0.6315169334411621, -0.974982500076294, 0.5302770733833313, -0.00427454337477684, 0.7057545781135559]),
 Row(id=2, features=[-0.01934131234884262, -0.7173973917961121, 0.29496514797210693, -1.16715407371521, 0.22981099784374237])]

# 物品特征向量V


[Row(id=0, features=[1.2744041681289673, -0.9583112597465515, 0.7699078917503357, 2.1497602462768555, 1.0788860321044922]),
 Row(id=1, features=[0.9104045629501343, -1.2607403993606567, 0.7037384510040283, 0.21806049346923828, 0.9687231183052063]),
 Row(id=2, features=[0.5538492202758789, -2.3204479217529297, 1.0750685930252075, -2.2940917015075684, 1.1120164394378662]),
 Row(id=3, features=[1.3332509994506836, -1.6890230178833008, 0.967412531375885, 0.5762561559677124, 1.3656255006790161])]



# 对于未知的用户物品推荐度预测结果
Row(user=0, item=2, prediction=0.16739553213119507)
Row(user=1, item=0, prediction=2.899648666381836)
Row(user=2, item=3, prediction=1.112522840499878)
Row(user=2, item=0, prediction=-1.371224284172058)

U0用户对所有物品的评分
[Row(item=[0, 3, 1, 2], rating=[3.8730804920196533, 3.0107336044311523, 1.9768474102020264, 0.16739553213119507])]

所有用户对物品V2的评分
[Row(user=[2, 1, 0], rating=[4.904191017150879, 3.9768621921539307, 0.16739553213119507])]

根据mode1分解的两个矩阵，对用户U2子集中3个物品，进行评分预测，然后与源数据对比
Row(item=[2, 3, 1], rating=[4.904191017150879, 1.112522840499878, 1.0625348091125488])

根据mode1分解的两个矩阵，对物品V0子集中3个用户，进行评分预测，然后与源数据对比
Row(user=[0, 1, 2], rating=[3.8730804920196533, 2.899648666381836, -1.371224284172058])
